In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('tracks_edited.csv', sep=',')

### Линейная регрессия

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [4]:
tags=np.unique(df.tag.values)

In [5]:
categorical_features=tags

In [6]:
one_hot = pd.get_dummies(df['tag'], prefix='tag')
df = pd.concat([df, one_hot], axis=1)
df = df.drop('tag', axis=1)

In [7]:
df.columns

Index(['artist_name', 'song_name', 'duration', 'playcount', 'listeners',
       'name_size', 'playcounts_per_listener', 'tag_2017', 'tag_2021',
       'tag_2022',
       ...
       'tag_surf rock', 'tag_synth funk', 'tag_synthpop', 'tag_synthwave',
       'tag_sza', 'tag_trap', 'tag_trip-hop', 'tag_tyler the creator',
       'tag_uh huh honey', 'tag_unique'],
      dtype='object', length=166)

Изменили формат категориальной переменной tag, чтобы было удобнее построить модель 

In [8]:
from sklearn.model_selection import train_test_split  # функция для деления 

# делим в пропорции 80 к 20
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=42)

y_train = df_train['playcount']
y_test = df_test['playcount']

X_train = df_train[['duration', 'listeners', 'name_size', 'playcounts_per_listener', 'tag_Hip-Hop',
       'tag_MySpotigramBot', 'tag_alternative', 'tag_dream pop', 'tag_folk',
       'tag_indie', 'tag_indie pop', 'tag_indie rock', 'tag_k-pop', 'tag_pop',
       'tag_rap', 'tag_rnb', 'tag_rock', 'tag_soul', 'tag_trap']]
X_test = df_test[['duration', 'listeners', 'name_size', 'playcounts_per_listener', 'tag_Hip-Hop',
       'tag_MySpotigramBot', 'tag_alternative', 'tag_dream pop', 'tag_folk',
       'tag_indie', 'tag_indie pop', 'tag_indie rock', 'tag_k-pop', 'tag_pop',
       'tag_rap', 'tag_rnb', 'tag_rock', 'tag_soul', 'tag_trap']]

In [12]:
from sklearn.linear_model import LinearRegression # подгрузили модель

model_regression = LinearRegression(fit_intercept=False)

# Обучили модель на тренировочной выборке 
model_regression.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [13]:
model_regression.coef_ # получили коэффициенты модели

array([-5.99705167e+05,  9.46982996e+00, -1.99735476e+04,  2.80707351e+05,
       -3.17202516e+04,  1.60847706e+05,  6.15030282e+05,  1.14423001e+06,
        1.20322045e+06,  7.14296898e+05,  1.12535994e+06,  1.06596079e+06,
       -4.40829714e+05,  9.07775717e+05, -7.08052099e+05,  1.01141184e+06,
       -2.66530098e+05, -4.72351923e+05,  5.78947212e+05])

In [14]:
y_pred=model_regression.predict(X_test)

In [15]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_pred, y_test)) 

2145764.6273004403

RMSE показал, что мы в среднем ошибаемся на 2,15 млн playcounts

### Random Forest  Classifier

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
model_rf=RandomForestRegressor(random_state=42)

In [18]:
model_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [19]:
y_pred=model_rf.predict(X_test)

In [20]:
np.sqrt(mean_squared_error(y_pred, y_test)) 

2016710.0850335972

RMSE показал, что мы в среднем ошибаемся на 2,02 млн playcounts для трэков

### Support Vector Machines 

In [21]:
from sklearn import svm

In [22]:
clf = svm.SVC(random_state=42)
clf.fit(X_train, y_train)

SVC(random_state=42)

In [23]:
y_pred=clf.predict(X_test)

In [24]:
np.sqrt(mean_squared_error(y_pred, y_test)) 

5360354.2731236555

RMSE показал, что мы в среднем ошибаемся на 3,55 млн playcounts для трэков

In [25]:
df.playcount.mean()

7817094.625211506

Итог: 
Мы попробовали отмасштабировать данные с помощью StandartScaler(), но это очень сильно ухудшило результат работы линейной регресси (с RMSE от 2 выросло до 8 миллионов), а на результаты моделей случайного леса и опорных векторов это не оказало значительного влияния, поэтому мы в конце решили этого не делать. 
При сравнении метрик мы обнаружили, что наименьшим значением RMSE обладает модель случайного леса, то есть из представленных трех она работает наиболее эффективно: в принципе ошибка в 2,02 миллионов это не очень критично (среднее значение-многомиллионное значение). 
Решили предсказывать именно значение playcount, а не listeners, так как, на наш взгляд, именно первая зависит от второй. Чем больше человек добавляют песню в плейлист, тем больше как минимум "официальных" слушателей у конкретного трека. 
В качестве метрики качества мы использовали RMSE, так как процентные метрики в случае многомиллионных показателей стримов не так репрезентативны, а возведение в квадрат разности (MSE) также не позваляет наглядно оценить результаты, они слишком большие.

 ### Random Forest  Classifier оптимизация

In [27]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [100, 200, 500],'criterion': ['mse', 'mae', 'friedman_mse', 'poisson'],
    'max_depth': [2, 3, 5],'min_samples_leaf': [1, 2, 3],'min_samples_split': [2, 3, 5],
    'max_features': [2, 3, 5]}
grid_search = GridSearchCV(model_rf, param_grid, cv=4)
grid_search.fit(X_train, y_train)
best_params=grid_search.best_params_

In [28]:
best_params

{'criterion': 'mse',
 'max_depth': 5,
 'max_features': 5,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 500}

In [29]:
best_model_rf=RandomForestRegressor(n_estimators=500, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=3, 
                                     max_features=5)
best_model_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, max_features=5, min_samples_leaf=3,
                      n_estimators=500)

In [30]:
y_pred=best_model_rf.predict(X_test)

In [31]:

np.sqrt(mean_squared_error(y_pred, y_test)) 

3526198.479929482

Прооптимизированный случайный лес представил большую ошибку, чем модель с дефолтными показателями 